In [1]:
import os
import numpy as np
import pandas as pd
import itertools
from itertools import combinations

In [3]:
measures = ['coh', 'pli', 'wpli', 'plv']
channel_names = ['E1', 'E2', 'F3', 'C3', 'O1', 'M2', 'EMG']
channel_indices = np.arange(0, 7, 1)
frequency_names = ['delta', 'theta', 'sigma', 'beta']
connectivity_measure = ['coh', 'plv', 'pli', 'wpli']

In [4]:
def channel_pairs(channels):
    pairs = []
    for i in range(len(channels)):
        for j in range(i + 1, len(channels)):
            pairs.append([channels[i], channels[j]])
    
    return pairs

In [5]:
patient_list  =  [  'P6 N2', 'P7 N1', 'P7 N2', 'P8 N1', 'P10 N1', 'P11 N1', 'P15 N1',
                  'P16 N1', 'P17 N1', 'P18 N1','P20 N1', 'P21 N1', 'P21 N2', 'P22 N1',
                  'P23 N1', 'P24 N1','P27 N1', 'P28 N1', 'P28 N2', 'P29 N2', 'P30 N1',
                  'P23 N2', 'P23 N3', 'P21 N3', 'P1 N1', 'P2 N1', 'P2 N2', 'P3 N1', 
                  'P3 N2', 'P4 N1', 'P4 N2', 'P5 N1', 'P6 N1'] 

genotype_human = {'P1': 'WT', 'P2': 'GAP', 'P3': 'GAP', 'P4': 'WT', 
                  'P5': 'GAP', 'P6': 'GAP', 'P7': 'GAP', 'P8' : 'WT',
                  'P9': 'GAP', 'P10': 'GAP', 'P11': 'WT', 'P12': 'WT',
                  'P13': 'GAP', 'P14': 'WT', 'P15': 'GAP', 'P16': 'GAP',
                  'P17': 'WT', 'P18': 'WT', 'P19': 'WT', 'P20': 'GAP',
                  'P21': 'WT', 'P22': 'GAP', 'P23': 'GAP', 'P24': 'WT',
                  'P25': 'WT', 'P26': 'GAP', 'P27': 'WT', 'P28': 'WT',
                  'P29': 'WT', 'P30': 'GAP'} 


In [6]:
mne_channel_comb = channel_pairs(channel_indices)

In [7]:
folder_dir = '/home/melissa/RESULTS/FINAL_MODEL/Human/Connectivity_MNE/'
results_dir = '/home/melissa/RESULTS/FINAL_MODEL/Human/Connectivity_MNE/dataframes/'
noise_path = '/home/melissa/PREPROCESSING/SYNGAP1/human_npy/harmonic_idx/'

In [12]:
os.chdir('/home/melissa/RESULTS/FINAL_MODEL/Human/Connectivity_MNE/dataframes/')
df = pd.read_csv('P6 N2_all_conn_measures.csv')

In [14]:
duplicate_columns = df.columns[df.columns.duplicated()]
duplicate_columns

Index([], dtype='object')

In [11]:
genotype_human = {'P1': 'WT', 'P2': 'GAP', 'P3': 'GAP', 'P4': 'WT', 
                  'P5': 'GAP', 'P6': 'GAP', 'P7': 'GAP', 'P8' : 'WT',
                  'P9': 'GAP', 'P10': 'GAP', 'P11': 'WT', 'P12': 'WT',
                  'P13': 'GAP', 'P14': 'WT', 'P15': 'GAP', 'P16': 'GAP',
                  'P17': 'WT', 'P18': 'WT', 'P19': 'WT', 'P20': 'GAP',
                  'P21': 'WT', 'P22': 'GAP', 'P23': 'GAP', 'P24': 'WT',
                  'P25': 'WT', 'P26': 'GAP', 'P27': 'WT', 'P28': 'WT',
                  'P29': 'WT', 'P30': 'GAP'} 

channel_labels = ['E1', 'E2', 'F3', 'C3', 'O1', 'M2']

patient_ls_df = []
for patient in patient_list:
    print(patient)
    patient_id = patient.split()[0]
    print(patient_id)
    noise_file = np.load(noise_path + patient + '_noise.npy')
    metric_ls = []
    for metric in connectivity_measure:
        print(metric + ' starting')
        for frequency in frequency_names:
            print(frequency + ' starting')
            folder_path = folder_dir + str(metric) + '/'
            patient_file = np.load(folder_path  + patient + '_' + frequency + '.npy')
            channel_combinations = list(itertools.product(channel_labels, repeat=2))
            frequency_ls_conn = []
            # Iterate through the connectivity matrix and print channel combinations
            for idx, value in enumerate(patient_file):
                if idx in noise_file:
                    pass
                else:
                    for i, conn_value in enumerate(value):
                        if conn_value != 0:
                            for key, value in genotype_human.items():
                                if key == patient_id:
                                    genotype = value
                                else:
                                    pass

                            chan_1 = channel_combinations[i][0]
                            chan_2 = channel_combinations[i][1]
                            connect_dict = {'Idx': [idx], 'Genotype': [genotype], 'Patient_ID': [patient],
                                            str(metric) + '_' + str(frequency) + '_' + chan_1 + '_' + chan_2: conn_value}
                            connect_df = pd.DataFrame(data = connect_dict)
                            frequency_ls_conn.append(connect_df)
            frequency_df = pd.concat(frequency_ls_conn, axis = 0)
            metric_ls.append(frequency_df)
    print('metric complete')
    patient_df = pd.concat(metric_ls, axis = 0)
    patient_df.to_csv(patient_results_path + (str(patient) + '_all_conn_measures.csv'))
    patient_ls_df.append(patient_df)
    print('patient complete')
    
all_patients = pd.concat(patient_ls_df)
all_patients.to_csv(results_dir + 'all_patients_all_conn_measures.csv')
print('dataframe saved')

P6 N2
P6
coh starting
delta starting
theta starting
sigma starting


KeyboardInterrupt: 

In [31]:
len(channel_combinations)

36

In [11]:
# Find all possible combinations of channels based on indices (including same-channel connections)
from itertools import combinations_with_replacement
channel_combinations = list(combinations_with_replacement(channel_indices, 2))

# Print the channel combinations using channel names
for index1, index2 in channel_combinations:
    channel1 = channel_names[index1]
    channel2 = channel_names[index2]
    print(f"Channel Combination: ({channel1}, {channel2})")

Channel Combination: (E1, E1)
Channel Combination: (E1, E2)
Channel Combination: (E1, F3)
Channel Combination: (E1, C3)
Channel Combination: (E1, O1)
Channel Combination: (E1, M2)
Channel Combination: (E1, EMG)
Channel Combination: (E2, E2)
Channel Combination: (E2, F3)
Channel Combination: (E2, C3)
Channel Combination: (E2, O1)
Channel Combination: (E2, M2)
Channel Combination: (E2, EMG)
Channel Combination: (F3, F3)
Channel Combination: (F3, C3)
Channel Combination: (F3, O1)
Channel Combination: (F3, M2)
Channel Combination: (F3, EMG)
Channel Combination: (C3, C3)
Channel Combination: (C3, O1)
Channel Combination: (C3, M2)
Channel Combination: (C3, EMG)
Channel Combination: (O1, O1)
Channel Combination: (O1, M2)
Channel Combination: (O1, EMG)
Channel Combination: (M2, M2)
Channel Combination: (M2, EMG)
Channel Combination: (EMG, EMG)


In [13]:
len(channel_combinations)

28

In [6]:
# Find all possible channel combinations
channel_combinations = list(combinations(channel_names, 2))

# Print the channel combinations
for combination in channel_combinations:
    chan_1 = combination[0]
    chan_2 = combination[1]
    break

In [7]:
channel_combinations = list(combinations(channel_indices, 2))
channel_combinations

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (3, 4),
 (3, 5),
 (3, 6),
 (4, 5),
 (4, 6),
 (5, 6)]

In [8]:
test = np.load(folder_dir + 'P1 N1_beta.npy')
test.shape

(1679, 36, 1)

In [9]:
for idx in test:
    print(idx.flatten())
    break

[0.         0.         0.         0.         0.         0.
 0.05459553 0.         0.         0.         0.         0.
 0.05459553 1.         0.         0.         0.         0.
 0.05459553 1.         1.         0.         0.         0.
 0.05459553 1.         1.         1.         0.         0.
 0.11881233 0.04644899 0.04644899 0.04644899 0.04644899 0.        ]


In [5]:
#conn_measure = 'coh'
#channel_names = ['E1', 'E2', 'F3', 'C3', 'O1', 'M2']
#channel_indices = np.arange(0, 6, 1)

# Create a sample array of functional connectivity values (replace this with your actual data)
#functional_connectivity_values = np.random.rand(36)

# Create lists to store channel pairs and their corresponding connectivity values


def func_conn_df_human(patient_id, array_idx, conn_measure, freq_band, channel_names, channel_indices):
    
    test_ls = []

    # Iterate through all combinations of channel pairs
    for i in range(len(channel_indices)):
        for j in range(i + 1, len(channel_indices)):
            func_dict = {channel_names[i] + '_' + channel_names[j] + '_' + conn_measure + '_' + str(freq_band): array_idx[i * len(channel_indices) + j]}
            func_df = pd.DataFrame(data = func_dict )
            test_ls.append(func_df)
    
    test_concat = pd.concat(test_ls, axis = 1)
    return test_concat

In [6]:
import pandas as pd

def func_conn_df_human(patient_id, array_idx, conn_measure, freq_band, channel_names, channel_indices):
    
    # Create an empty DataFrame with columns for each channel pair
    func_df = pd.DataFrame(columns=[channel_names[i] + '_' + channel_names[j] + '_' + conn_measure + '_' + str(freq_band)
                                    for i in range(len(channel_indices))
                                    for j in range(i + 1, len(channel_indices))])

    # Populate the DataFrame with connectivity values
    for i in range(len(channel_indices)):
        for j in range(i + 1, len(channel_indices)):
            col_name = channel_names[i] + '_' + channel_names[j] + '_' + conn_measure + '_' + str(freq_band)
            func_df[col_name] = [array_idx[i * len(channel_indices) + j]]

    return func_df

# Example usage:
patient_id = "patient1"
array_idx = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
conn_measure = "corr"
freq_band = "alpha"
channel_names = ['E1', 'E2', 'F3', 'C3', 'O1', 'M2']
channel_indices = list(range(len(channel_names)))

result_df = func_conn_df_human(patient_id, array_idx, conn_measure, freq_band, channel_names, channel_indices)
print(result_df)


IndexError: list index out of range

In [17]:
measure_ls = []
for measure in measures:
    conn_folder = folder_dir + measure + '/'
    patient_ls = []
    for file in os.listdir(conn_folder):
        if file.startswith('.ipy'):
            pass
        else:
            print(file)
            splits = file.split('_')
            frequency_band = splits[1].split('.')[0]
            patient = splits[0].split(' ')[0]
            conn_file = np.load(conn_folder + file)
            conn_indices = []
            for conn_idx in conn_file:
                func_concat = func_conn_df_human(patient_id = patient, 
                               array_idx = conn_idx.flatten(), conn_measure = measure, freq_band = frequency_band, 
                              channel_names = channel_names, channel_indices = channel_indices)
                print(func_concat)
                conn_indices.append(func_concat)
            conn_concat = pd.concat(conn_indices)
            patient_ls.append(conn_concat)
        patient_concat = pd.concat(patient_ls)
        measure_ls.append(patient_concat)
    measure_concat = pd.concat(measure_ls)
    measure_concat.to_csv(results_dir + str(measure) + '.csv')

P10 N1_sigma.npy


ValueError: If using all scalar values, you must pass an index

In [10]:
conn_concat

,Patient_ID,E1_E2_coh_delta,Patient_ID,E1_F3_coh_delta,Patient_ID,E1_C3_coh_delta,Patient_ID,E1_O1_coh_delta,Patient_ID,E1_M2_coh_delta,...,Patient_ID,F3_O1_coh_delta,Patient_ID,F3_M2_coh_delta,Patient_ID,C3_O1_coh_delta,Patient_ID,C3_M2_coh_delta,Patient_ID,O1_M2_coh_delta
0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,...,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0
0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,...,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0
0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,...,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0
0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,...,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0
0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,...,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,...,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0
0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,...,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0
0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,...,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0
0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0,...,P6,0.0,P6,0.0,P6,0.0,P6,0.0,P6,0.0
